In [1]:
import numpy as np
import warnings
from munch import Munch
import itertools

In [2]:
hidden_states = ['a','b','c']
emit_states = ['A','C']
hmm_transition = {}
for i in hidden_states:
    for j in hidden_states:
        hmm_transition[i,j] = 1/3

emit_mat = np.array([
    [.8,.2],
    [.5,.5],
    [.2,.8]
])
hmm_emit = {}
for i in range(3):
    for j in range(2):
        hmm_emit[hidden_states[i],emit_states[j]] = emit_mat[i,j].item()
        
hmm_startprob = {}
for i in hidden_states:
    hmm_startprob[i] = 1/3

hmm = Munch(states = hidden_states, emits = emit_states, tprob = hmm_transition, eprob = hmm_emit, initprob = hmm_startprob)

In [3]:
def update_fun(r,k , r_past):
    '''
    m1^t = tau^t_a = a OR tau^{t-1}_a #tracks if state a has happend yet 
    m2^t = [1- (1 - tau^t_a) AND c)] AND m2^{t-1} = [tau^t_a or (1 - c)] AND m2^{t-1} #tracks if the arrival time of a is before c
    k is the current state
    r is the auxillary state. a 2-tuple. r = (m1,m2)
    '''
    m1 = (k == 'a') or r_past[0]
    m2 = (m1 or (not k == 'c')) and r_past[1]

    return int(r == (m1,m2))

def init_fun(k, r):
    '''
    initial "prob" of r = (m1,m2) from k. is just indicator
    '''
    m1 = k == 'a'
    m2 = not k == 'c'

    return int(r == (m1,m2))
    
def cst_fun(r, sat):
    '''
    Constraint is a boolean emissions of the final auxillary state. In this case, is just m1^T: ie. tau_a >= tau_b for all time.
    '''
    return int(r[1] == sat) 

In [4]:
cst = Munch(aux_size = 2, update_fun = update_fun, init_fun = init_fun, cst_fun = cst_fun)

In [5]:
def mv_Viterbi(obs, hmm, cst, sat = True):
    '''
    Does Viterbii with intermediate variables. In this version, the constraint is included as an binary emission at the last time. 
    This formulation allows us to easily with inference in the case where the constraint is satisfied or not.
    
    obs: list of observed emissions
    hmm: Munch object containig our hmm object
    cast: Munch object containing our constraint (cst) object
    sat. Boolean determining whether the constraint is ture or not
    
    '''
    aux_space = list(itertools.product([True, False], repeat=cst.aux_size)) #constraint.aux_size
    val = {}

    for k in hmm.states:
        for r in aux_space:
            val[0,k,r] = cst.init_fun(k,r)*hmm.initprob[k]*hmm.eprob[k,obs[0]]
            
    ix_tracker = {}
    
    #Forward: compute value function and generate index
    for t in range(1,len(obs)):
        for k in hmm.states:
            for r in aux_space:
                max_val = -1 # set to dummy variable. will do brute-force search for max
                argmax = None #initialize argmax for ix_tracker
                for j in hmm.states:
                    for s in aux_space:
                        curr_val = val[t-1,j,s]*hmm.tprob[j,k]*cst.update_fun(r,j,s)
                        if curr_val > max_val:
                            max_val = curr_val
                            argmax = (j,s)
                if t == (len(obs)-1): #ie. at the last time we add in the constraint
                    val[t,k,r] = max_val*hmm.eprob[k,obs[t]]*cst.cst_fun(r,sat)
                else:
                    val[t,k,r] = max_val*hmm.eprob[k,obs[t]]
                ix_tracker[t-1,k,r] = argmax
    
    #Backward: compute the values of the optimal sequence
    max_val = -1
    best_state = None
    for k in hmm.states:
        for r in aux_space:
            curr_val = val[len(obs) - 1,k,r]
            if curr_val > max_val:
                max_val = curr_val
                best_state = (k,r)
    opt_augstate = [best_state]            
    opt_state = [best_state[0]]
    
    for t in range(len(obs) -1):
        best_state = ix_tracker[len(obs) - 2 -t,best_state[0], best_state[1]]
        opt_augstate = [best_state] + opt_augstate #append at the front
        opt_state = [best_state[0]] + opt_state

    return(opt_augstate, opt_state)

In [6]:
obs = ['C','C','A','C','A','C']

In [7]:
opt_aug, opt_state = mv_Viterbi(obs, hmm, cst, sat = True)

In [8]:
opt_state

['b', 'b', 'a', 'c', 'a', 'c']

In [76]:
obs = ['A','C','A','C']
aux_space = list(itertools.product([True, False], repeat=2)) #constraint.aux_size
val = {}
for k in hmm.states:
    for r in aux_space:
        val[0,k,r] = cst.init_fun(k,r)*hmm.initprob[k]*hmm.eprob[k,obs[0]]
        
ix_tracker = {}

#Forward: compute value function and generate index
for t in range(1,len(obs)):
    for k in hmm.states:
        for r in aux_space:
            max_val = -1 # set to dummy variable. will do brute-force search for max
            argmax = None #initialize argmax for ix_tracker
            for j in hmm.states:
                for s in aux_space:
                    curr_val = val[t-1,j,s]*hmm.tprob[j,k]*cst.update_fun(r,j,s)
                    if curr_val > max_val:
                        max_val = curr_val
                        argmax = (j,s)
            if t == (len(obs)-1): #ie. at the last time we add in the constraint
                val[t,k,r] = max_val*hmm.eprob[k,obs[t]]*cst.cst_fun(r,True)
            else:
                val[t,k,r] = max_val*hmm.eprob[k,obs[t]]
            ix_tracker[t-1,k,r] = argmax

#Backward: compute the values of the optimal sequence
max_val = -1
best_state = None
for k in hmm.states:
    for r in aux_space:
        curr_val = val[len(obs) - 1,k,r]
        if curr_val > max_val:
            max_val = curr_val
            best_state = (k,r)
opt_augstate = [best_state]            
opt_state = [best_state[0]]

for t in range(len(obs) -1):
    best_state = ix_tracker[len(obs) - 2 -t,best_state[0], best_state[1]]
    opt_augstate = [best_state] + opt_augstate #append at the front
    opt_state = [best_state[0]] + opt_state

In [36]:
val[0,'a',(True,False)]

0.0

In [ ]:
def viterbi(obs, hmm,  cst, satisfied = True)
    T = len(obs)
    
    V = {}
    for k in hmm.states:
        for 
    